# Colab Setup

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

work_dir = '/content/drive/MyDrive/ML_Iocchi_Assignments/Assignment_1'
dataset_dir = os.path.join(work_dir, 'Dataset')

In [6]:
dataset_path = os.path.join(dataset_dir, '0_combined_r2_data.csv')

try:
    dataset = pd.read_csv(dataset_path, delimiter=';')
    dataset.columns = dataset.columns.str.strip()
    print(dataset.head())
except FileNotFoundError:
    print(f"Error: File not found at {dataset_path}")
except pd.errors.EmptyDataError:
    print(f"Error: The file at {dataset_path} is empty.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

      j0     j1  cos(j0)  cos(j1)  sin(j0)  sin(j1)   ee_x   ee_y  ee_qw  \
0 -0.044 -0.008    0.999    1.000   -0.044   -0.008  0.210 -0.010  1.000   
1 -0.062 -0.011    0.998    1.000   -0.061   -0.011  0.210 -0.014  0.999   
2 -0.126 -0.015    0.992    1.000   -0.126   -0.015  0.208 -0.028  0.998   
3 -0.226 -0.051    0.974    0.999   -0.224   -0.051  0.203 -0.053  0.990   
4 -0.364 -0.082    0.935    0.997   -0.356   -0.082  0.193 -0.083  0.975   

   ee_qz  
0 -0.026  
1 -0.036  
2 -0.070  
3 -0.138  
4 -0.221  


# Model

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.kernel_approximation import RBFSampler

from sklearn.metrics import mean_squared_error, r2_score

Data Setup

In [8]:
X = dataset[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values

y_pos    = dataset[['ee_x', 'ee_y']]
y_orient = dataset[['ee_qw', 'ee_qz']]


# Scaling
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y_pos = StandardScaler()
y_pos_scaled = scaler_y_pos.fit_transform(y_pos)

scaler_y_orient = StandardScaler()
y_orient_scaled = scaler_y_orient.fit_transform(y_orient)

# training and test set division
X_pos_train, X_pos_test, y_pos_train, y_pos_test = train_test_split(X_scaled, y_pos_scaled, test_size=0.3, random_state=77)
X_orient_train, X_orient_test, y_orient_train, y_orient_test = train_test_split(X_scaled, y_orient_scaled, test_size=0.3, random_state=77)


In [10]:
from sklearn.model_selection import ParameterGrid

def train_with_grid_search(X_train, X_test, y_train, y_test, scaler, param_grid, random_state=23):
    best_params = None
    best_mse = float('inf')
    best_r2 = float('-inf')

    for params in ParameterGrid(param_grid):
        gamma = params['gamma']
        n_components = params['n_components']

        # RBF Transformation
        rbf_sampler = RBFSampler(gamma=gamma, n_components=n_components, random_state=random_state)
        X_train_rbf = rbf_sampler.fit_transform(X_train)
        X_test_rbf = rbf_sampler.transform(X_test)

        # Linear Regression
        model = LinearRegression()
        model.fit(X_train_rbf, y_train)

        # Model predictions
        y_pred = model.predict(X_test_rbf)
        y_pred_original_scale = scaler.inverse_transform(y_pred)

        # Valutazione del modello
        mse = mean_squared_error(scaler.inverse_transform(y_test), y_pred_original_scale)
        r2 = r2_score(scaler.inverse_transform(y_test), y_pred_original_scale)

        print(f"Params: gamma={gamma}, n_components={n_components}, MSE={mse:.9f}, R2={r2:.9f}")

        if mse < best_mse:
            best_mse = mse
            best_r2 = r2
            best_params = params

    print("\nBest Parameters:")
    print(f"gamma: {best_params['gamma']}, n_components: {best_params['n_components']}")
    print(f"Best MSE: {best_mse:.9f}, Best R2: {best_r2:.9f}")
    return best_params, best_mse, best_r2


param_grid = {
    'gamma': [0.001, 0.1, 0.5],
    'n_components': [50, 100, 200]
}

best_pos_params, best_pos_mse, best_pos_r2 = train_with_grid_search(X_pos_train, X_pos_test, y_pos_train, y_pos_test, scaler_y_pos, param_grid)
best_orient_params, best_orient_mse, best_orient_r2 = train_with_grid_search(X_orient_train, X_orient_test, y_orient_train, y_orient_test, scaler_y_orient, param_grid)


Params: gamma=0.001, n_components=50, MSE=0.000000185, R2=0.999977801
Params: gamma=0.001, n_components=100, MSE=0.000000180, R2=0.999978379
Params: gamma=0.001, n_components=200, MSE=0.000000178, R2=0.999978589
Params: gamma=0.1, n_components=50, MSE=0.000005046, R2=0.999398967
Params: gamma=0.1, n_components=100, MSE=0.000000207, R2=0.999975222
Params: gamma=0.1, n_components=200, MSE=0.000000179, R2=0.999978508
Params: gamma=0.5, n_components=50, MSE=0.000180775, R2=0.978953006
Params: gamma=0.5, n_components=100, MSE=0.000003542, R2=0.999574748
Params: gamma=0.5, n_components=200, MSE=0.000000197, R2=0.999976347

Best Parameters:
gamma: 0.001, n_components: 200
Best MSE: 0.000000178, Best R2: 0.999978589
Params: gamma=0.001, n_components=50, MSE=0.000046206, R2=0.999901035
Params: gamma=0.001, n_components=100, MSE=0.000001855, R2=0.999996037
Params: gamma=0.001, n_components=200, MSE=0.000001691, R2=0.999996391
Params: gamma=0.1, n_components=50, MSE=0.000379068, R2=0.999193927
Pa

# Best models

Position

In [9]:
# RBF Transformation
rbf_pos_sampler = RBFSampler(gamma=0.001, n_components=200, random_state=23)
X_pos_train_rbf = rbf_pos_sampler.fit_transform(X_pos_train)
X_pos_test_rbf = rbf_pos_sampler.transform(X_pos_test)

# Linear Regression
model_pos = LinearRegression()
model_pos.fit(X_pos_train_rbf, y_pos_train)

# Predictions
# Model predictions
y_pos_pred = model_pos.predict(X_pos_test_rbf)
y_pos_pred_original_scale = scaler_y_pos.inverse_transform(y_pos_pred)

mse_pos = mean_squared_error(scaler_y_pos.inverse_transform(y_pos_test), y_pos_pred_original_scale)
print(f'{mse_pos:.10f}')

0.0000001782


Orientation

In [10]:
# RBF Transformation
rbf_orient_sampler = RBFSampler(gamma=0.001, n_components=200, random_state=23)
X_orient_train_rbf = rbf_orient_sampler.fit_transform(X_orient_train)
X_orient_test_rbf = rbf_orient_sampler.transform(X_orient_test)

# Linear Regression
model_orient = LinearRegression()
model_orient.fit(X_orient_train_rbf, y_orient_train)

# Predictions
# Model predictions
y_orient_pred = model_orient.predict(X_orient_test_rbf)
y_orient_pred_original_scale = scaler_y_orient.inverse_transform(y_orient_pred)

mse_orient = mean_squared_error(scaler_y_orient.inverse_transform(y_orient_test), y_orient_pred_original_scale)
print(f'{mse_orient:.6f}')

0.000002


#### Evaluation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Results from the grid search
gamma_values = [0.001, 0.1, 0.5]
n_components_values = [50, 100, 200]
mse_results = [
    [0.000010, 0.000010, 0.000010],  # gamma = 0.001
    [0.000013, 0.000010, 0.000010],  # gamma = 0.1
    [0.000915, 0.000012, 0.000010],  # gamma = 0.5
]

# Convert results to numpy array for easier manipulation
mse_array = np.array(mse_results)

# Heatmap of MSE values
plt.figure(figsize=(10, 6))
sns.heatmap(mse_array, annot=True, fmt=".6f", xticklabels=n_components_values, yticklabels=gamma_values, cmap="viridis")
plt.title("MSE Heatmap")
plt.xlabel("Number of Components")
plt.ylabel("Gamma")
plt.show()

# Line plots for MSE vs. n_components for each gamma
plt.figure(figsize=(10, 6))
for i, gamma in enumerate(gamma_values):
    plt.plot(n_components_values, mse_array[i], marker='o', label=f"Gamma = {gamma}")

plt.title("MSE vs. Number of Components")
plt.xlabel("Number of Components")
plt.ylabel("MSE")
plt.legend()
plt.grid(True)
plt.show()


# Jacobian Evaluation

In [12]:
# Compute the analytical Jacobian matrix for a 2-DOF robot
def jacobian_analytical(j0, j1, L1=0.1, L2=0.1):

    # Partial derivations
    dx_dj0 = -L1 * np.sin(j0) - L2 * np.sin(j0 + j1)
    dx_dj1 = -L2 * np.sin(j0 + j1)
    dy_dj0 = L1 * np.cos(j0) + L2 * np.cos(j0 + j1)
    dy_dj1 = L2 * np.cos(j0 + j1)

    # Jacobian matrix construction
    jacobian = np.array([
        [dx_dj0, dx_dj1],
        [dy_dj0, dy_dj1],
    ])
    return jacobian

In [13]:
def compute_features(j0, j1, rbf_sampler, scaler_features):
    # Feature vector with joint angles and their trigonometric values
    features = np.array([[j0, j1, np.cos(j0), np.sin(j0), np.cos(j1), np.sin(j1)]])

    # Scale features
    features_scaled = scaler_features.transform(features)

    # Apply RBF transformation
    features_scaled_rbf = rbf_sampler.transform(features_scaled)

    return pd.DataFrame(features_scaled_rbf)

In [14]:
def learned_jacobian(model, j0, j1, rbf_sampler, scaler_X, scaler_y, delta=1e-5):
    # Compute features for base features and perturbed ones
    base = compute_features(j0, j1, rbf_sampler, scaler_X)
    perturbed_j0 = compute_features(j0 + delta, j1, rbf_sampler, scaler_X)
    perturbed_j1 = compute_features(j0, j1 + delta, rbf_sampler, scaler_X)

    # Predictions
    base_out = model.predict(base).flatten()[:2]
    d_outputs_dj0 = (model.predict(perturbed_j0).flatten()[:2] - base_out) / delta
    d_outputs_dj1 = (model.predict(perturbed_j1).flatten()[:2] - base_out) / delta

    # Jacobian matrix construction
    return np.column_stack([d_outputs_dj0, d_outputs_dj1])

In [15]:
def compare_jacobians(model, j0, j1, rbf_sampler, scaler_X, scaler_y):
    # Compute Jacobians
    J_analytic = jacobian_analytical(j0, j1)
    J_learned = learned_jacobian(model, j0, j1, rbf_sampler, scaler_X, scaler_y)

    # Calculate the Frobenius norm
    frobenius_norm = np.linalg.norm(J_learned - J_analytic)

    # Display results
    print("\nAnalytical Jacobian:\n", J_analytic)
    print("\nLearned Jacobian:\n", J_learned)
    print(f"\nFrobenius norm of difference: {frobenius_norm:.6f}")

    return J_analytic, J_learned, frobenius_norm

# Example usage
theta1_example, theta2_example = 0.1, -0.3
print("Comparing Jacobians for Model 1 (targets: ee_x, ee_y):")
compare_jacobians(
    model_pos, theta1_example, theta2_example, rbf_pos_sampler, scaler_X, scaler_y_pos
)

Comparing Jacobians for Model 1 (targets: ee_x, ee_y):

Analytical Jacobian:
 [[0.00988359 0.01986693]
 [0.19750707 0.09800666]]

Learned Jacobian:
 [[ 1.60144409 -1.08979875]
 [ 2.42888345  0.57616271]]

Frobenius norm of difference: 2.995346


(array([[0.00988359, 0.01986693],
        [0.19750707, 0.09800666]]),
 array([[ 1.60144409, -1.08979875],
        [ 2.42888345,  0.57616271]]),
 2.995345809272796)